In [ ]:
import ee
import geemap
import folium

# Initialisieren Sie Earth Engine
ee.Initialize(project='ee-dkloer01')

# Basis-Karteneinstellungen
base_change = [{
    'featureType': 'all',
    'stylers': [{
        'saturation': -100
    }, {
        'lightness': 45
    }]
}]

# GHSL Bevölkerungsdaten laden
image1995 = ee.Image('JRC/GHSL/P2023A/GHS_POP/1995')
image2005 = ee.Image('JRC/GHSL/P2023A/GHS_POP/2005')
image2015 = ee.Image('JRC/GHSL/P2023A/GHS_POP/2015')
image2025 = ee.Image('JRC/GHSL/P2023A/GHS_POP/2025')


# Visualisierungsparameter für die Bevölkerungsdichte
population_count_vis = {
    'min': 0.0,
    'max': 100.0,
    'palette': ['000004', '320A5A', '781B6C', 'BB3654', 'EC6824', 'FBB41A', 'FCFFA4'],
}

# Dubai-Geometrie laden (ersetzen Sie dies durch Ihr FeatureCollection)
# Hier wird ein Beispiel-FeatureCollection verwendet, Sie müssen es anpassen
shapefile_path = "C:/Users/dkloe/Downloads/Ashkinadze_Kloer/Dubai_Shapes/Dubai.shp"
dubai = geemap.shp_to_ee(shapefile_path)

# Extrahiere die Geometrie aus der FeatureCollection
dubai_geometry = dubai.geometry()

# Auf Dubai zuschneiden und Maske anwenden
image1995 = image1995.clip(dubai.geometry()).updateMask(image1995.gt(0))
image2005 = image2005.clip(dubai.geometry()).updateMask(image2005.gt(0))
image2015 = image2015.clip(dubai.geometry()).updateMask(image2015.gt(0))
image2025 = image2025.clip(dubai.geometry()).updateMask(image2025.gt(0))

# Karte erstellen
m = geemap.Map(center=[25.2048, 55.2708], zoom=10)  # Dubai zentrieren

# Layer zur Karte hinzufügen
m.addLayer(image1995, population_count_vis, 'Bevölkerungsdichte, 1995')
m.addLayer(image2005, population_count_vis, 'Bevölkerungsdichte, 2005')
m.addLayer(image2015, population_count_vis, 'Bevölkerungsdichte, 2015')
m.addLayer(image2025, population_count_vis, 'Bevölkerungsdichte, 2025')

# Legende erstellen und hinzufügen
legend_dict = {
    "0-14,28": "000004",
    "14,28-28,56": "320A5A",
    "28,56-42,84": "781B6C",
    "42,84-57,12": "BB3654",
    "57,12-71,4": "EC6824",
    "71,4-85,68": "FBB41A",
    "85,68-100": "FCFFA4"
}

style = {
    "position": "fixed",
    "z-index": "9999",
    "border": "2px solid grey",
    "background-color": "rgba(255, 255, 255, 0.8)",
    "border-radius": "10px",
    "padding": "5px",
    "font-size": "14px",
    "bottom": "20px",
    "right": "5px"
}

m.add_legend(
    title="Einwohner pro km²", legend_dict=legend_dict, draggable=False, style=style
)


# Karte anzeigen
m


# # Starte den Export
# task = ee.batch.Export.image.toDrive(
#     image=image1995.visualize(**population_count_vis).updateMask(image1995.mask()),
#     description='Population_1995',
#     folder='GIS1_Pop',
#     scale=30,
#     region=dubai_geometry
# )
# task.start()


# # Karte als HTML-Datei exportieren
# m.to_html("./dubai_population_map.html")




Map(center=[25.2048, 55.2708], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

In [4]:
image1995 = ee.Image('JRC/GHSL/P2023A/GHS_POP/2025')
population_band = image1995.select('population_count')

# Get the population count value at the ROI.
population_value = population_band.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=dubai,
    scale=100  # Scale in meters
).get('population_count').getInfo()

print(f'Population count at the specified location: {population_value}')

Population count at the specified location: 672.3790283203125


In [10]:
html = geemap.create_legend(
    title="NLCD Land Cover Type",
    builtin_legend="NLCD",
    draggable=False,
    position="bottomright",
)

In [11]:
widget = geemap.show_html(html)
widget

HTML(value='<!DOCTYPE html>\n<html lang="en">\n    <head> </head>\n    <body>\n        <div\n            id="m…

In [12]:
widget.close()

In [13]:
legend_dict = {
    "10 Trees": "006400",
    "20 Shrubland": "ffbb22",
    "30 Grassland": "ffff4c",
    "40 Cropland": "f096ff",
    "50 Built-up": "fa0000",
    "60 Barren / sparse vegetation": "b4b4b4",
    "70 Snow and ice": "f0f0f0",
    "80 Open water": "0064c8",
    "90 Herbaceous wetland": "0096a0",
    "95 Mangroves": "00cf75",
    "100 Moss and lichen": "fae6a0",
}

In [14]:
geemap.create_legend(
    title="ESA Land Cover Type",
    legend_dict=legend_dict,
    draggable=False,
    output="ESA_legend.html",
)

In [18]:
m = geemap.Map()
m.add_basemap("ESA Worldcover 2021")

style = {
    "position": "fixed",
    "z-index": "9999",
    "border": "2px solid grey",
    "background-color": "rgba(255, 255, 255, 0.8)",
    "border-radius": "10px",
    "padding": "5px",
    "font-size": "14px",
    "bottom": "20px",
    "right": "5px",
}

m.add_legend(
    title="ESA Land Cover Type", legend_dict=legend_dict, draggable=False, style=style
)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [20]:
m = geemap.Map()
m.add_basemap("ESA Worldcover 2021")
m.add_legend(title="ESA Land Cover Type", builtin_legend="ESA_WorldCover")
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [27]:
m = geemap.Map(
    center=[40, -100],
    zoom=4,
    draw_control=False,
    measure_control=False,
    scale_control=False,
)
m.split_map(left_layer="ESA Worldcover 2021", right_layer="NLCD 2019 CONUS Land Cover")
m.add_legend(
    title="ESA Land Cover Type",
    builtin_legend="ESA_WorldCover",
    draggable=False,
    position="bottomleft",
    style={"bottom": "5px"},
)
m.add_legend(
    title="NLCD Land Cover Type",
    builtin_legend="NLCD",
    draggable=False,
    position="bottomright",
)
m

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…